In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler

#####solar power
#loda data
f_names=['jindosolar_'+str(i) for i in range(2016,2018+1)]
data_solar=[]
for f in f_names:
    df=pd.read_csv('data/solar/'+f+'.csv',encoding = 'ISO-8859-1').values
    df=df[:,4:].flatten()
    print(f,df.shape)
    data_solar.append(df)
data_solar=np.concatenate(data_solar,axis=0)[:,np.newaxis]
#data=data_solar.copy();
print(data_solar.shape)
print(np.min(data_solar),np.max(data_solar))

###weather
#loda data
tmpidx=pd.date_range(start='201601010000',end='201812312300',freq="H")
tmpst=pd.DataFrame(np.nan, index=tmpidx, columns=["idx"])

f_names=['jindoweather_'+str(i) for i in range(2016,2018+1)]
df_weather=pd.DataFrame()
for f in f_names:
    df=pd.read_csv('data/weather/'+f+'.csv',encoding = 'ISO-8859-1')
    df=df.iloc[:,[1,2,3,4,5,6,7,8,9,11,19,22]]
    df_weather=pd.concat((df_weather,df),axis=0)
col=['temp','rain','ws','wd','humi','prs','ptemp','gprs','sun','sight','gtemp']
df_weather.columns=['date']+col 
#df.columns.values[0]='date'
df_weather['date'] = df_weather['date'].astype(str)
df_weather['date'] = pd.to_datetime(df_weather['date'],format="%Y-%m-%d %H:%M")
df_weather=df_weather.set_index('date')
df_weather=df_weather.combine_first(tmpst)
del df_weather['idx']

df_weather = df_weather[col]
print(df_weather.shape)

#fill missing data
df_weather['sun'] = df_weather['sun'].fillna(0)
df_weather['rain'] = df_weather['rain'].fillna(0)
wind=pd.DataFrame({'wd': np.cos(np.pi*df_weather["wd"]/360)})#wind=pd.DataFrame({'풍향(sin)': np.sin(np.pi*weather["풍향(16방위)"]/360),'풍향(cos)': np.cos(np.pi*weather["풍향(16방위)"]/360)})
df_weather=df_weather.drop("wd",axis=1)
df_weather=pd.concat((df_weather,wind),axis=1)
df_weather=df_weather.interpolate(method='linear')#linear interpolation
df_weather=df_weather[col] 
data_weather=df_weather.values
print(data_weather.shape)
#matching time index: first day 1h.,,,.last day 23h
data_solar=data_solar[:-1]
data_weather=data_weather[1:]
print(data_solar.shape)
print(data_weather.shape)

#####Data Preprocessing
data_weather_wfore=[]
for i in range(11):
    data_weather_wfore.append(data_weather[:-24,i])
    data_weather_wfore.append(data_weather[24:,i])#forecasting
data_weather_wfore=np.stack(data_weather_wfore,axis=1)
data_wfore=np.concatenate((data_solar[:-24],data_weather_wfore),axis=1);print(data_wfore.shape)
data_wofore=np.concatenate((data_solar[:-24],data_weather[:-24]),axis=1);print(data_wofore.shape)

def generate_data(data,wofore):
    ########################################
    n_steps_in=72#24
    n_steps_out=24#1
    output_dir='data'
    n_f=1
    ########################################
    x_offsets = np.sort(np.concatenate((np.arange(-n_steps_in+1, 1, 1),)))
    y_offsets = np.sort(np.arange(1, n_steps_out+1, 1))
    num_samples=len(data)
    x, y = [], []
    # t is the index of the last observation.
    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
    for t in range(min_t, max_t):
        x_t = data[t + x_offsets, ...]
        y_t = data[t + y_offsets, ...]
        x.append(x_t)
        y.append(y_t)
    x = np.stack(x, axis=0)
    y = np.stack(y, axis=0)

    num_test = round(num_samples * 0.2)
    num_train = round(num_samples * 0.6)
    num_val = num_samples - num_test - num_train#(15767, 5256, 5256)
    x_train, y_train = x[:num_train], y[:num_train]
    x_val, y_val = (
        x[num_train: num_train + num_val],
        y[num_train: num_train + num_val],
    )
    x_test, y_test = x[-num_test:], y[-num_test:]
    for cat in ["train", "val", "test"]:
        _x, _y = locals()["x_" + cat], locals()["y_" + cat]
        print(cat, "x: ", _x.shape, "y:", _y.shape)
        if wofore:
            cat=cat+'_wofore'
        np.savez_compressed(
            os.path.join(output_dir, "%s.npz" % cat),
            x=_x,
            y=_y,
            x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
            y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
        )
    
#generate_data(data=data_wofore,wofore=True)
#generate_data(data=data_wfore,wofore=False)

data_wofore=pd.DataFrame(data_wofore)
col=['solar','temp','rain','ws','wd','humi','prs','ptemp','gprs','sun','sight','gtemp']
data_wofore.columns=col 
#data_wofore.to_csv('data/data_wofore.csv',index = False)

jindosolar_2016 (8784,)
jindosolar_2017 (8760,)
jindosolar_2018 (8760,)
(26304, 1)
0 953136
(26304, 11)
(26304, 11)
(26303, 1)
(26303, 11)
(26279, 23)
(26279, 12)
